<a href="https://colab.research.google.com/github/Nithin2211/News-Summariser/blob/main/News_summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install exact package versions and spacy model
!pip install -q -U pip setuptools==68.0.0 wheel
!pip install -q transformers sentence-transformers torch spacy sumy rouge-score scikit-learn requests matplotlib wordcloud
!pip install -q git+https://github.com/csurfer/rake-nltk.git
!python -m spacy download en_core_web_sm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 804.0/804.0 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 39.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 21.6 MB/

In [ ]:
import os
import sys
import json
import time
import requests
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import spacy
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer

import torch
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer, util

from rake_nltk import Rake
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import precision_score, recall_score, f1_score
from rouge_score import rouge_scorer

from wordcloud import WordCloud

# NLTK setup
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab') # Added this line

# SpaCy setup
nlp = spacy.load('en_core_web_sm')

# Set NewsAPI environment key
NEWSAPI_KEY = input("Enter your NewsAPI key: ").strip()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
CATEGORIES = ['technology', 'sports', 'business', 'health', 'entertainment']
ARTICLES_PER_CAT = 30
DATA_CACHE = 'newsapi_cache.json'

def fetch_news(category, api_key, page_size=30, retries=3):
    url = f'https://newsapi.org/v2/top-headlines?category={category}&pageSize={page_size}&apiKey={api_key}&language=en'
    for attempt in range(retries):
        response = requests.get(url)
        if response.status_code == 200:
            return response.json()['articles']
        time.sleep(2 + attempt)
    return []

# Load cached data if exists & valid
if os.path.exists(DATA_CACHE):
    with open(DATA_CACHE, 'r') as f:
        cached = json.load(f)
    if all(cat in cached for cat in CATEGORIES):
        news_data = cached
    else:
        news_data = {}
else:
    news_data = {}

# Fetch missing categories with retry
for cat in CATEGORIES:
    if cat not in news_data or len(news_data[cat]) < ARTICLES_PER_CAT:
        fetched = fetch_news(cat, NEWSAPI_KEY, ARTICLES_PER_CAT)
        news_data[cat] = fetched

# Save to cache
with open(DATA_CACHE, 'w') as f:
    json.dump(news_data, f)
print("Fetched articles per category:", {cat: len(news_data[cat]) for cat in CATEGORIES})


In [ ]:
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Lowercase, remove non-alpha characters
    text = ''.join([ch.lower() if ch.isalpha() or ch.isspace() else ' ' for ch in str(text)])
    tokens = nltk.word_tokenize(text)
    tokens = [t for t in tokens if t not in stop_words]
    stemmed = [stemmer.stem(t) for t in tokens]
    lemmatized = [lemmatizer.lemmatize(t) for t in stemmed]
    return ' '.join(lemmatized)

# Preprocess articles
for cat in CATEGORIES:
    for article in news_data[cat]:
        # Ensure all parts are strings before concatenation
        title = article.get('title') or ''
        description = article.get('description') or ''
        content = article.get('content') or ''
        article['cleaned_text'] = preprocess_text(f"{title}. {description}. {content}")

In [ ]:
import torch
# Extractive Summarization: TextRank via sumy
def textrank_summary(text, num_sentences=3):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = TextRankSummarizer()
    summary = summarizer(parser.document, num_sentences)
    return ' '.join(str(sentence) for sentence in summary)

# Abstractive Summarization: BART
bart_summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=0 if torch.cuda.is_available() else -1)

def bart_summary(text, min_length=40, max_length=130):
    try:
        summary = bart_summarizer(text, min_length=min_length, max_length=max_length)[0]['summary_text']
    except Exception as e:
        summary = ""
    return summary

# Hybrid: TextRank then BART
def hybrid_summary(text):
    intermediate = textrank_summary(text, num_sentences=5)
    return bart_summary(intermediate)

# Apply summarization for first article per category
hybrid_summaries = {}
for cat in CATEGORIES:
    article = news_data[cat][0]
    text = article['cleaned_text']
    extractive = textrank_summary(text, num_sentences=3)
    abstractive = bart_summary(text, min_length=40, max_length=130)
    hybrid = hybrid_summary(text)
    article['summaries'] = {'extractive': extractive, 'abstractive': abstractive, 'hybrid': hybrid}
    hybrid_summaries[cat] = hybrid

In [ ]:
# RAKE keyword extraction
def extract_rake_keywords(text, top_n=10):
    rake = Rake()
    rake.extract_keywords_from_text(text)
    return rake.get_ranked_phrases()[:top_n]

# TF-IDF keyword extraction over all articles
all_texts = [art['cleaned_text'] for cat in CATEGORIES for art in news_data[cat]]
vectorizer = TfidfVectorizer(max_features=50, stop_words='english')
tfidf_matrix = vectorizer.fit_transform(all_texts)
feature_names = vectorizer.get_feature_names_out()

def extract_tfidf_keywords(text, top_n=10):
    response = vectorizer.transform([text])
    sorted_indices = np.argsort(response.toarray()[0])[::-1][:top_n]
    return [feature_names[i] for i in sorted_indices]

# Store top keywords for articles
for cat in CATEGORIES:
    for article in news_data[cat]:
        text = article['cleaned_text']
        article['keywords'] = {
            'rake': extract_rake_keywords(text, 10),
            'tfidf': extract_tfidf_keywords(text, 10)
        }


In [ ]:
def get_ner(text):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

# Example NER for first article each category
ner_samples = {}
for cat in CATEGORIES:
    article = news_data[cat][0]
    ner_entities = get_ner(article['cleaned_text'])
    article['ner'] = ner_entities
    ner_samples[cat] = ner_entities


In [ ]:
# Prepare labeled dataset from NewsAPI (based on categories)
texts = []
labels = []
for cat in CATEGORIES:
    for article in news_data[cat]:
        txt = article['cleaned_text']
        if txt:
            texts.append(txt)
            labels.append(cat)

# TF-IDF features
tfidf = TfidfVectorizer(max_features=200, stop_words='english')
X = tfidf.fit_transform(texts)
y = np.array(labels)

# Split for baseline
idx = np.arange(len(y))
np.random.seed(42)
train_idx = idx[:int(0.7*len(y))]
test_idx = idx[int(0.7*len(y)):]

clf = LogisticRegression(max_iter=300)
clf.fit(X[train_idx], y[train_idx])

y_pred = clf.predict(X[test_idx])

precision = precision_score(y[test_idx], y_pred, average='weighted')
recall = recall_score(y[test_idx], y_pred, average='weighted')
f1 = f1_score(y[test_idx], y_pred, average='weighted')
classification_result = {
    'precision': precision, 'recall': recall, 'f1': f1
}
print("Classification (TF-IDF+LogReg):", classification_result)


In [ ]:
# Content-based using Sentence-Transformer embeddings
embedder = SentenceTransformer('all-MiniLM-L6-v2')
user_profile_text = 'technology sports health entertainment business'
user_embedding = embedder.encode(user_profile_text)
article_embeddings = [embedder.encode(art['cleaned_text']) for cat in CATEGORIES for art in news_data[cat]]

# Content-based scores: cosine similarity to user embedding
content_scores = util.cos_sim(user_embedding, article_embeddings).flatten().numpy()

# Collaborative filtering (SVD matrix factorization)
# The 'ratings' matrix correctly represents (categories, articles)
ratings = np.zeros((len(CATEGORIES), len(article_embeddings)))
for i, cat in enumerate(CATEGORIES):
    all_articles_flat_list = [article_obj for cat2 in CATEGORIES for article_obj in news_data[cat2]]
    ratings[i, :] = [1.0 if article_obj in news_data[cat] else 0 for article_obj in all_articles_flat_list]

svd = TruncatedSVD(n_components=5, random_state=42)
svd.fit(ratings) # Fit SVD model to the ratings matrix

# svd.components_ provides the latent features of articles, shape (n_components, num_articles)
# Sum or average these components to get a single collaborative score per article
collab_scores = svd.components_.mean(axis=0) # This will yield a (num_articles,) array

# Hybrid recommendation: weighted sum
hybrid_scores = 0.6 * content_scores + 0.4 * collab_scores
rec_indices = np.argsort(hybrid_scores)[::-1][:5]
top_recs = [all_texts[i] for i in rec_indices]

In [ ]:
# ROUGE evaluation between TextRank, BART, and Hybrid summaries
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_results = {}

for cat in CATEGORIES:
    art = news_data[cat][0]
    ref = art['cleaned_text']  # Use preprocessed article as reference
    smry = art['summaries']['hybrid']
    scores = scorer.score(ref, smry)
    rouge_results[cat] = {k: v.fmeasure for k, v in scores.items()}

# Print classification metrics (already calculated)
print("ROUGE results by category:", rouge_results)
print("Classification metrics:", classification_result)


In [ ]:
# WordCloud for all articles
wc_text = ' '.join([art['cleaned_text'] for cat in CATEGORIES for art in news_data[cat]])
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(wc_text)

plt.figure(figsize=(10,5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title("Word Cloud of News Corpus")
plt.show()

# Distribution plots: number of articles per category
counts = [len(news_data[cat]) for cat in CATEGORIES]
plt.figure(figsize=(7,4))
plt.bar(CATEGORIES, counts, color='skyblue')
plt.title('News Articles per Category')
plt.xlabel('Category')
plt.ylabel('Count')
plt.show()


In [ ]:
print("\n------ Academic-style Results Summary ------")
print("Number of articles fetched per category:")
for cat in CATEGORIES:
    print(f"- {cat}: {len(news_data[cat])}")

print("\nExample Hybrid Summary (Technology):")
print(news_data['technology'][0]['summaries']['hybrid'])

print("\nExtracted Keywords (Technology, RAKE & TF-IDF):")
print("RAKE:", news_data['technology'][0]['keywords']['rake'])
print("TF-IDF:", news_data['technology'][0]['keywords']['tfidf'])

print("\nNamed Entities (Technology):")
print(news_data['technology'][0]['ner'])

print("\nText Classification Result (Weighted):")
print(classification_result)

print("\nTop 5 Personalized Recommendations (hybrid):")
for idx, rec in enumerate(top_recs):
    print(f"{idx+1}. {rec[:250]}...")  # Preview rec summaries

print("\nROUGE metrics (Hybrid summaries):")
for cat in CATEGORIES:
    print(f"{cat}: {rouge_results[cat]}")

print("\nClassification metrics (Precision/Recall/F1):")
print(classification_result)

print("\n------ End of Output ------")
